## 1. Setup & Data Loading

**Dataset:** BTS Airline On-Time Performance (2022-2024)  
**Source:** Bureau of Transportation Statistics  
**Key Metric:** ArrDelay15 (1 = arrival ≥15 minutes late, 0 = on-time)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
pd.set_option('display.max_columns', None)

In [ ]:
# Load dataset
df = pd.read_csv('../DATA/airline_delays_2022_2024.csv')

print(f"Total records: {len(df):,}")
print(f"Columns: {len(df.columns)}")

In [ ]:
# Validate required columns
required_cols = ['CarrierName', 'ArrDelay15', 'Cancelled']
missing_cols = [col for col in required_cols if col not in df.columns]

if missing_cols:
    print(f"⚠️  Missing required columns: {missing_cols}")
    print(f"Available columns: {list(df.columns)}")
else:
    print("✅ All required columns present")

## 2. Data Exploration

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
# Missing values
missing = df.isnull().sum()
missing_pct = (missing / len(df)) * 100
missing_df = pd.DataFrame({'Missing Count': missing, 'Percentage': missing_pct})
missing_df[missing_df['Missing Count'] > 0].sort_values('Missing Count', ascending=False)

In [ ]:
# Airlines in dataset
print(df['CarrierName'].value_counts())
print(f"\nTotal airlines: {df['CarrierName'].nunique()}")

In [ ]:
# Date range
print(f"From: {df['Year'].min()}-{df['Month'].min():02d}")
print(f"To: {df['Year'].max()}-{df['Month'].max():02d}")

In [ ]:
# Overall rates
print(f"Overall delay rate: {df['ArrDelay15'].mean() * 100:.2f}%")
print(f"Overall cancellation rate: {df['Cancelled'].mean() * 100:.2f}%")

## 3. Data Preparation

In [ ]:
# Filter to completed flights (non-cancelled)
df_completed = df[df['Cancelled'] == 0].copy()

print(f"Total flights: {len(df):,}")
print(f"Completed flights: {len(df_completed):,}")
print(f"Cancellation rate: {(df['Cancelled'].mean() * 100):.2f}%")

## 4. Compute Airline Metrics

In [ ]:
# Delay rate by airline
delay_metrics = df_completed.groupby('CarrierName').agg(
    total_flights=('ArrDelay15', 'count'),
    delayed_flights=('ArrDelay15', 'sum')
).reset_index()

delay_metrics['delay_rate_pct'] = (
    delay_metrics['delayed_flights'] / delay_metrics['total_flights'] * 100
)

delay_metrics.sort_values('delay_rate_pct', ascending=False)

In [ ]:
# Cancellation rate by airline
cancellation_metrics = df.groupby('CarrierName').agg(
    total_flights=('Cancelled', 'count'),
    cancelled_flights=('Cancelled', 'sum')
).reset_index()

cancellation_metrics['cancellation_rate_pct'] = (
    cancellation_metrics['cancelled_flights'] / cancellation_metrics['total_flights'] * 100
)

cancellation_metrics.sort_values('cancellation_rate_pct', ascending=False)

In [ ]:
# Average delay magnitude (for delayed flights)
df_delayed = df_completed[df_completed['ArrDelay15'] == 1].copy()

# Check if ArrDelayMinutes column exists
if 'ArrDelayMinutes' in df_delayed.columns:
    avg_delay_metrics = df_delayed.groupby('CarrierName').agg(
        avg_delay_minutes=('ArrDelayMinutes', 'mean'),
        median_delay_minutes=('ArrDelayMinutes', 'median')
    ).reset_index()
    avg_delay_metrics.sort_values('avg_delay_minutes', ascending=False)
else:
    print("⚠️  ArrDelayMinutes column not found - skipping delay magnitude analysis")
    avg_delay_metrics = pd.DataFrame({'CarrierName': df['CarrierName'].unique(), 'avg_delay_minutes': 0})

In [ ]:
# Comprehensive scorecard
scorecard = delay_metrics[['CarrierName', 'total_flights', 'delay_rate_pct']].copy()
scorecard = scorecard.merge(cancellation_metrics[['CarrierName', 'cancellation_rate_pct']], on='CarrierName')

# Only add avg_delay_minutes if the column exists
if 'avg_delay_minutes' in avg_delay_metrics.columns:
    scorecard = scorecard.merge(avg_delay_metrics[['CarrierName', 'avg_delay_minutes']], on='CarrierName')

scorecard = scorecard.sort_values('delay_rate_pct', ascending=False)
scorecard = scorecard.round(2)
print(scorecard.to_string(index=False))

## 5. Visualizations

In [ ]:
# Delay rate by airline
plt.figure(figsize=(12, 6))
scorecard_sorted = scorecard.sort_values('delay_rate_pct', ascending=True)

bars = plt.barh(scorecard_sorted['CarrierName'], scorecard_sorted['delay_rate_pct'], 
                color='steelblue', edgecolor='black')

worst_idx = scorecard_sorted['delay_rate_pct'].idxmax()
bars[list(scorecard_sorted.index).index(worst_idx)].set_color('crimson')

plt.xlabel('Delay Rate (%)', fontsize=12, fontweight='bold')
plt.ylabel('Airline', fontsize=12, fontweight='bold')
plt.title('Delay Rate by Carrier (2022-2024)', fontsize=14, fontweight='bold', pad=20)

for i, (idx, row) in enumerate(scorecard_sorted.iterrows()):
    plt.text(row['delay_rate_pct'] + 0.3, i, f"{row['delay_rate_pct']:.1f}%", va='center', fontsize=10)

plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Cancellation rate by airline
plt.figure(figsize=(12, 6))
scorecard_sorted = scorecard.sort_values('cancellation_rate_pct', ascending=True)

plt.barh(scorecard_sorted['CarrierName'], scorecard_sorted['cancellation_rate_pct'], 
         color='coral', edgecolor='black')

plt.xlabel('Cancellation Rate (%)', fontsize=12, fontweight='bold')
plt.ylabel('Airline', fontsize=12, fontweight='bold')
plt.title('Cancellation Rate by Carrier (2022-2024)', fontsize=14, fontweight='bold', pad=20)

for i, (idx, row) in enumerate(scorecard_sorted.iterrows()):
    plt.text(row['cancellation_rate_pct'] + 0.05, i, f"{row['cancellation_rate_pct']:.2f}%", va='center', fontsize=10)

plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Delay vs Cancellation scatter
plt.figure(figsize=(10, 7))

plt.scatter(scorecard['delay_rate_pct'], scorecard['cancellation_rate_pct'], 
            s=200, alpha=0.6, c='steelblue', edgecolors='black', linewidth=1.5)

for idx, row in scorecard.iterrows():
    plt.annotate(row['CarrierName'], (row['delay_rate_pct'], row['cancellation_rate_pct']),
                 xytext=(5, 5), textcoords='offset points', fontsize=9)

plt.xlabel('Delay Rate (%)', fontsize=12, fontweight='bold')
plt.ylabel('Cancellation Rate (%)', fontsize=12, fontweight='bold')
plt.title('Delay Rate vs. Cancellation Rate (2022-2024)', fontsize=14, fontweight='bold', pad=20)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 6. Compare to BTS Official Rankings

In [ ]:
# Display your ranking
print("YOUR COMPUTED RANKING (worst to best):")
for idx, row in scorecard.iterrows():
    rank = list(scorecard.index).index(idx) + 1
    print(f"{rank}. {row['CarrierName']:<25} {row['delay_rate_pct']:.2f}%")

## 7. Key Findings Summary

**Instructions:** Compare your results with the official BTS Annual Rankings at:  
https://www.bts.gov/topics/airlines-and-airports/annual-airline-time-rankings-2003-2024

Document any differences and explain why they might occur (e.g., different time periods, filtering criteria, or calculation methods).